## Create The ONNX File

In [1]:
from special_neurons import get_most_negative_sets
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import re

model_name = 'EleutherAI/pythia-160m'
model_name = 'EleutherAI/pythia-70m'
# model_name = 'EleutherAI/gpt-neo-1.3B'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
model_og = AutoModelForCausalLM.from_pretrained(model_name).to(device)
most_neg = get_most_negative_sets(model_og)
most_neg[0].prev_layer_name, most_neg[0].linear_layer_name

/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.2.1) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


('gpt_neox.layers.0.mlp', 'gpt_neox.layers.0.mlp.dense_h_to_4h')

In [19]:
# Our max_norm is none
print(model_og.gpt_neox.embed_in.max_norm, model_og.gpt_neox.embed_in.norm_type, model_og.gpt_neox.embed_in.scale_grad_by_freq)

None 2.0 False


In [21]:
embd_matrix = model_og.gpt_neox.embed_in.weight
embd_matrix.shape

torch.Size([50304, 512])

In [28]:
def get_input_after_tokenizer(inp: str):
		tokenizer = AutoTokenizer.from_pretrained(model_name)
		inp_out = tokenizer(inp, return_tensors="pt")
		print(inp_out)
		inputs = inp_out['input_ids'].squeeze(0)
		# print(inputs, inputs.shape)
		# TODO: IDK ABOUT WHATS GOING ON W/ TOKEN SIZE VS Vocab Size
		# Vocab size is 50204 and inp size is 50304
		one_hot = torch.zeros((inputs.shape[0], 50304), dtype=torch.int)
		for i in range(inputs.shape[0]):
			one_hot[i, inputs[i]] = 1
		# one_hot[inputs['input_ids'][0, 0]] = 1
		return one_hot#, inp_out['attention_mask']
get_input_after_tokenizer('hello world')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


{'input_ids': tensor([[25521,  1533]]), 'attention_mask': tensor([[1, 1]])}


tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], dtype=torch.int32)

In [29]:
inps_one_hot = get_input_after_tokenizer("hello world")
inps_one_hot.shape, inps_one_hot.nonzero()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


{'input_ids': tensor([[25521,  1533]]), 'attention_mask': tensor([[1, 1]])}


(torch.Size([2, 50304]),
 tensor([[    0, 25521],
         [    1,  1533]]))

In [51]:
N_TOKENS = inps_one_hot.shape[0]
attention_mask = torch.ones((1, N_TOKENS), dtype=torch.int)


class FixedAttentionMask(torch.nn.Module):
  	
    def __init__(self, *args, **kwargs) -> None:
       super().__init__(*args, **kwargs)
       self.attn = model_og.gpt_neox.layers[0].attention

    def forward(self, x):
      # TODO: IDK IF THIS IS RIGHT for POSITION IDS or ATTENTION MASK
      return self.attn(x, attention_mask=attention_mask, position_ids= torch.arange(N_TOKENS).unsqueeze(0))

embed_linear = torch.nn.Linear(embd_matrix.shape[0], embd_matrix.shape[1], bias=False)
embed_linear.weight = torch.nn.Parameter(embd_matrix.T)

# G
# TODO: add residuals?
model_sel = torch.nn.Sequential(
    # model_og.gpt_neox.embed_in,
    embed_linear,
    # model_og.gpt_neox.emb_dropout, # we have p = 0.0 and thus useless
    model_og.gpt_neox.layers[0].input_layernorm,
    FixedAttentionMask(),
    # TODO: VERIFY THIS JAZZ
)

model_sel

Sequential(
  (0): Linear(in_features=50304, out_features=512, bias=False)
  (1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  (2): FixedAttentionMask(
    (attn): GPTNeoXAttention(
      (rotary_emb): GPTNeoXRotaryEmbedding()
      (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
      (dense): Linear(in_features=512, out_features=512, bias=True)
      (attention_dropout): Dropout(p=0.0, inplace=False)
    )
  )
)

In [52]:
model_sel[0]

Linear(in_features=50304, out_features=512, bias=False)

In [47]:
inps_one_hot.shape, embed_linear.weight.shape

(torch.Size([2, 50304]), torch.Size([50304, 512]))

In [55]:
inps_one_hot = inps_one_hot.float()
embed_linear.forward(inps_one_hot[0])
# model_sel(inps_one_hot)[0]

AttributeError: 'Tensor' object has no attribute 'astype'

In [ ]:

torch.onnx.export(model_sel, torch.randn(1, 1).to(device),
                  'model_sel.onnx', verbose=True, opset_version=12)

## Use Mariboupy

In [ ]:
import sys
import numpy as np

## %
# Path to Marabou folder if you did not export it

# TODO: this is a hack, fix it
# TODO: add Marabou installation details to readme
# sys.path.append('/home/lev/code/research/softmax_bound/bounding-softmax/Marabou')
from maraboupy import Marabou